In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


In [ ]:
data = pd.read_csv('../../datas/Store Sales/train.csv')
holidays = pd.read_csv('../../datas/Store Sales/holidays_events.csv')
oil_price = pd.read_csv('../../datas/Store Sales/oil.csv')
transactions = pd.read_csv('../../datas/Store Sales/transactions.csv')


In [ ]:
data = data.dropna(ignore_index=True)
holidays = holidays.dropna(ignore_index=True)
oil_price = oil_price.dropna(ignore_index=True)
transactions = transactions.dropna(ignore_index=True)

In [ ]:
data['date'] = pd.to_datetime(data['date'])
transactions['date'] = pd.to_datetime(transactions['date'])
oil_price['date'] = pd.to_datetime(oil_price['date'])
holidays['date'] = pd.to_datetime(holidays['date'])

In [ ]:
data = data[data['date'].dt.year == 2017]
data = data[data['date'].dt.month.isin([6, 7, 8])]
data = data.rename(columns={'onpromotion': 'on_promotion'})
data

In [ ]:
oil_price = oil_price[oil_price['date'].dt.year == 2017]
oil_price = oil_price[oil_price['date'].dt.month.isin([6, 7, 8])]
oil_price = oil_price.rename(columns={'dcoilwtico': 'oil_price'})
oil_price

In [ ]:
holidays

EDA

In [ ]:
data.groupby(['store_nbr'])['sales'].sum().sort_values(ascending=False)[:5]

EDA on single store

In [ ]:
store_44 = data[data['store_nbr'] == 44].drop(columns='store_nbr')
store_44 = store_44.reset_index(drop=True)

In [ ]:
store_44['day'] = store_44['date'].dt.day

In [ ]:
def transform_value(value):
    if value in [-1, True]:
        return False
    elif value is False:
        return True
    return value

In [ ]:
store_44 = pd.merge(store_44, oil_price, on='date', how='left')
store_44['has_oil_price'] = store_44['oil_price'].isna()
store_44['oil_price'] = store_44['oil_price'].fillna(0)
store_44['has_oil_price'] = store_44['has_oil_price'].apply(transform_value)

In [ ]:
store_44

In [ ]:
is_holiday = pd.merge(store_44, holidays, on='date', how='left')
is_holiday = is_holiday[['type', 'locale', 'locale_name', 'description', 'transferred']]
is_holiday

In [ ]:
is_holiday[['type', 'locale', 'locale_name', 'description', 'transferred']] = is_holiday[['type', 'locale', 'locale_name', 'description', 'transferred']].fillna(-1)
is_holiday

In [ ]:
is_holiday['is_holiday'] = is_holiday['transferred'].apply(transform_value)
store_44['is_holiday'] = is_holiday['is_holiday']

In [ ]:
store_44

In [ ]:
# Plot using Plotly
fig = px.line(store_44, x='date', y='sales', color='family', title='Sales Over Time for Store 44')

# Update layout
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Sales',
    title='Sales Over Time for Store 44',
    legend_title='Family'
)

# Show the plot
fig.show()

In [ ]:
# Plot using Plotly
fig = px.line(store_44, x='date', y='oil_price', title='Oil Price Over Time')

# Update layout
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Oil Price',
    title='Oil Price Over Time'
)

# Show the plot
fig.show()

In [ ]:
categories = store_44.value_counts('family')
categories, len(categories)

EDA on all stores

In [243]:
stores = data.reset_index(drop=True)
stores['day'] = stores['date'].dt.day

stores = pd.merge(stores, oil_price, on='date', how='left')
stores['has_oil_price'] = stores['oil_price'].isna()
stores['oil_price'] = stores['oil_price'].fillna(0)
stores['has_oil_price'] = stores['has_oil_price'].apply(transform_value)

In [244]:
is_holiday = pd.merge(stores, holidays, on='date', how='left')
is_holiday = is_holiday[['type', 'locale', 'locale_name', 'description', 'transferred']]
is_holiday[['type', 'locale', 'locale_name', 'description', 'transferred']] = is_holiday[['type', 'locale', 'locale_name', 'description', 'transferred']].fillna(-1)
is_holiday['is_holiday'] = is_holiday['transferred'].apply(transform_value)
stores['is_holiday'] = is_holiday['is_holiday']

In [245]:
stores

,id,date,store_nbr,family,sales,on_promotion,day,oil_price,has_oil_price,is_holiday
0,2865456,2017-06-01,1,AUTOMOTIVE,6.000,0,1,48.32,True,False
1,2865457,2017-06-01,1,BABY CARE,0.000,0,1,48.32,True,False
2,2865458,2017-06-01,1,BEAUTY,1.000,0,1,48.32,True,False
3,2865459,2017-06-01,1,BEVERAGES,2250.000,9,1,48.32,True,False
4,2865460,2017-06-01,1,BOOKS,3.000,0,1,48.32,True,False
...,...,...,...,...,...,...,...,...,...,...
135427,3000883,2017-08-15,9,POULTRY,438.133,0,15,47.57,True,False
135428,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,15,47.57,True,False
135429,3000885,2017-08-15,9,PRODUCE,2419.729,148,15,47.57,True,False
135430,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,15,47.57,True,False
